In [1]:
from scipy import optimize
from bingo.evaluation.fitness_function import VectorBasedFunction
import numpy as np
import sys
from numpy import array

In [9]:
number_of_eqps_values = 21
eqps_values_np = np.array([ i*0.0015 for i in range(number_of_eqps_values) ])
P_desired = np.array([[1, -0.5, -0.5],
                    [-0.5, 1, -0.5],
                    [-0.5, -0.5, 1]])
dataloc = 'processed_data/vm_1_bingo_format.txt'
pre_data = np.loadtxt(dataloc)
print(pre_data)
training_data = []
eqpses = []
for row in pre_data:
    if np.isnan(row[0]) == False:
        training_data.append(row[:3])
        eqpses.append(row[3])
training_data = np.array(training_data)
eqpses = np.array(eqpses)
#print(eqpses)
print(training_data)

def mapping_(x):

    # mapping = lambda X_0: array([[x[0], x[1], x[2]],
    #    [x[3], x[4], x[5]],
    #    [x[6], x[7], x[8]]]) + (X_0) * (array([[x[9], x[10], x[11]],
    #    [x[12], x[12], x[13]],
    #    [x[14], x[15], x[16]]]))
    # mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], x[3], x[4] ],  
    #     [x[3] , x[6], x[5]],
    #     [x[4], x[5], x[7]]]))#*(10 + 990*X_0)
    # mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], 0, 0 ],  
    #     [0 , x[6], 0],
    #     [0, 0, x[7]]]))
    mapping = lambda X_0: (x[1]*X_0 + x[0])*np.eye(3)
    
    mapping_matrices = np.array([mapping(e) for e in eqpses])
    P_mapped = np.transpose(mapping_matrices,(0, 2, 1)) @ P_desired @ mapping_matrices
    back_calculated_yield_stresses = []
    for i,stresses in enumerate(training_data):
        #print('here we go', np.transpose(training_data[i]), P_mapped[i], training_data[i])
        #back_calculated_yield_stresses.append(np.transpose(training_data[i]) @ P_mapped[i] @ training_data[i])
        back_calculated_yield_stresses.append(P_mapped[i] @ training_data[i])
        #print(P_mapped[i] @ training_data[i])
    #print(back_calculated_yield_stresses)

    #error = np.ones(len(training_data)) - back_calculated_yield_stresses
    error = training_data - np.array(back_calculated_yield_stresses)
    #print(error)
    return error.flatten()



# ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],  
#        [-0.8733965 , -1.02045703, -0.37967495],
#        [-0.86253049, -1.02066836, -0.38872487]]))


[[ 2.1132486540518718e+00 -7.8867513459481291e+00  5.7735026918962582e+00
   0.0000000000000000e+00]
 [ 4.2053648215632249e+00 -1.5694635178436775e+01  1.1489270356873554e+01
   1.0000000000000000e-02]
 [ 6.2974809890745780e+00 -2.3502519010925425e+01  1.7205038021850850e+01
   2.0000000000000000e-02]
 [ 8.3895971565859320e+00 -3.1310402843414074e+01  2.2920805686828146e+01
   2.9999999999999999e-02]
 [ 1.0481713324097285e+01 -3.9118286675902716e+01  2.8636573351805442e+01
   4.0000000000000001e-02]
 [ 1.2573829491608638e+01 -4.6926170508391365e+01  3.4352341016782738e+01
   5.0000000000000003e-02]
 [ 1.4665945659119993e+01 -5.4734054340880014e+01  4.0068108681760037e+01
   5.9999999999999998e-02]
 [ 1.6758061826631348e+01 -6.2541938173368671e+01  4.5783876346737337e+01
   7.0000000000000007e-02]
 [ 1.8850177994142697e+01 -7.0349822005857305e+01  5.1499644011714629e+01
   8.0000000000000002e-02]
 [ 2.0942294161654051e+01 -7.8157705838345947e+01  5.7215411676691915e+01
   8.999999999999

In [3]:
x0 = [-0.83111601, -0.46657978, -0.06323608, -0.83507571, -0.45586107, -0.0702321, -0.84231627, -0.46026723, -0.05982535, 
      67.83825554, 11.92019618, 22.12241255, 67.75485734, 11.34285823, 22.7518725, 68.42081951, 11.30870116, 22.22108227
]
x0 = [ 0.00500253*2,0.00500253*2,-0.87240378, -1.01069406, -0.3899734, -0.8733965 , -1.02045703, -0.37967495 ]
x0 = [ 0, 1]
mapping_(x0)
#results = optimize.minimize(mapping_, )

array([   2.11324865,   -7.88675135,    5.77350269, ...,   41.31823768,
       -154.20176232,  112.88352463])

In [4]:
results = optimize.root(mapping_, x0=x0, method='lm', options={"xtol": 1e-16, "gtol": 1e-16, "maxiter": 100000})
results

   cov_x: array([[ 1.36371366e-08, -3.24614035e-08],
       [-3.24614035e-08,  2.07050419e-07]])
    fjac: array([[ 1.08161330e+04,  5.10834900e+00,  1.30750387e-03, ...,
         9.95445626e-03, -3.71505366e-02,  2.71960802e-02],
       [ 1.69575535e+03,  2.19766728e+03, -1.00840837e-03, ...,
        -7.67734402e-03,  2.86522380e-02, -2.09748939e-02]])
     fun: array([-1.59623252e-08,  5.95722103e-08, -4.36098846e-08, ...,
        1.07185919e-07, -4.00023254e-07,  2.92837356e-07])
    ipvt: array([1, 2], dtype=int32)
 message: 'The relative error between two consecutive iterates is at most 0.000000'
    nfev: 43
     qtf: array([1.16867814e-08, 4.49119794e-07])
  status: 2
 success: True
       x: array([ 8.16496584e-01, -2.03960018e-08])

In [5]:
np.set_printoptions(threshold=sys.maxsize)
mapping_(results.x)

array([-1.59623252e-08,  5.95722103e-08, -4.36098846e-08, -2.96640348e-08,
        1.10707692e-07, -8.10436518e-08, -4.12753129e-08,  1.54041569e-07,
       -1.12766251e-07, -5.07961513e-08,  1.89573829e-07, -1.38777676e-07,
       -5.82265614e-08,  2.17304503e-07, -1.59077931e-07, -6.35665351e-08,
        2.37233550e-07, -1.73667011e-07, -6.68160745e-08,  2.49361015e-07,
       -1.82544923e-07, -6.79751793e-08,  2.53686856e-07, -1.85711656e-07,
       -6.70438496e-08,  2.50211102e-07, -1.83167231e-07, -6.40220961e-08,
        2.38933723e-07, -1.74911619e-07, -5.89099045e-08,  2.19854769e-07,
       -1.60944850e-07, -5.17072678e-08,  1.92974156e-07, -1.41266895e-07,
       -4.24142002e-08,  1.58292011e-07, -1.15877782e-07, -3.10307051e-08,
        1.15808191e-07, -8.47774686e-08, -1.75567791e-08,  6.55228263e-08,
       -4.79660116e-08, -1.99240446e-09,  7.43580131e-09, -5.44338263e-09,
        1.56623798e-08, -5.84527697e-08,  4.27904183e-08,  3.54076306e-08,
       -1.32142986e-07,  

In [6]:
np.set_printoptions(precision=32)
results.x

array([ 8.164965840114106e-01, -2.039600180638302e-08])

In [7]:
# mapping = lambda X_0: array([[-0.83111601, -0.46657978, -0.06323608],
#        [-0.83507571, -0.45586107, -0.07023218],
#        [-0.84231627, -0.46026723, -0.05982535]]) + (X_0) * (array([[67.83825554, 11.92019618, 22.12241255],
#        [67.75485734, 11.34285823, 22.75187253],
#        [68.42081951, 11.30870116, 22.22108227]]))
# Q = mapping(0)
# Q